## 1. Import Libraries và Khởi tạo Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col, explode, split, trim, current_timestamp, row_number, 
    when, regexp_replace, monotonically_increasing_id, dense_rank, udf, collect_list
)
from pyspark.sql.window import Window
import os

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session với cấu hình tối ưu cho memory
spark = (
    SparkSession.builder.appName("Load_Data_To_Gold_Tables")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "1536m")
    .config("spark.executor.cores", "2")
    .config("spark.memory.fraction", "0.6")
    .config("spark.memory.storageFraction", "0.3")
    .config("spark.sql.shuffle.partitions", "50")
    .config("spark.default.parallelism", "50")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.shuffle.spill.compress", "true")
    .config("spark.shuffle.compress", "true")
    .config("spark.rdd.compress", "true")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # ===== Cấu hình MinIO =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("Spark Session đã được khởi tạo!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")


25/12/06 15:30:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Session đã được khởi tạo!
Spark Master: spark://spark-master:7077
Application ID: app-20251206153054-0002


## 2. Đọc Dữ Liệu từ Result Model và Lọc Records Chưa Xử Lý

In [2]:
# Đọc dữ liệu từ bảng result_multi_model
df_result = spark.table("nessie.silver_tables.result_multi_model")

print(f"Tổng số records trong result_multi_model: {df_result.count()}")

# Kiểm tra xem Post table đã có dữ liệu chưa
try:
    df_existing_posts = spark.table("nessie.gold_result_model_multi_task.Post")
    existing_post_ids = [row.postID for row in df_existing_posts.select("postID").distinct().collect()]
    print(f"Tìm thấy {len(existing_post_ids)} posts đã được xử lý trước đó")
    
    # Lọc ra các posts chưa được xử lý
    if existing_post_ids:
        df_result_new = df_result.filter(~col("postID").isin(existing_post_ids))
        print(f"Số lượng posts mới cần xử lý: {df_result_new.count()}")
        
        if df_result_new.count() == 0:
            print("\nKhông có posts mới để xử lý. Tất cả dữ liệu đã được load vào Gold tables.")
            print("Nếu muốn load lại, hãy xóa dữ liệu trong Gold tables trước.")
        else:
            df_result = df_result_new
    else:
        print("Không có posts nào trong Gold tables, sẽ xử lý toàn bộ dữ liệu")
        
except Exception as e:
    print(f"Gold tables chưa có dữ liệu hoặc chưa tồn tại: {e}")
    print("Sẽ xử lý toàn bộ dữ liệu từ result_multi_model")

print("\nSchema:")
df_result.printSchema()
print("\nSample data:")
df_result.show(5, truncate=False)


Tổng số records trong result_multi_model: 3295
Tìm thấy 0 posts đã được xử lý trước đó
Không có posts nào trong Gold tables, sẽ xử lý toàn bộ dữ liệu

Schema:
root
 |-- postID: string (nullable = true)
 |-- timePublish: timestamp (nullable = true)
 |-- description_Normalized: string (nullable = true)
 |-- Label_NER: string (nullable = true)
 |-- Label_Topic: string (nullable = true)
 |-- Label_Intent: string (nullable = true)
 |-- likeCount: integer (nullable = true)
 |-- commentCount: integer (nullable = true)
 |-- shareCount: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


Sample data:


+----------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------+----------------+---------+------------+----------+--------+--------------------------+--------------------------+
|postID          |timePublish        |description_Normalized                                                                                                                                                                                                                                                         

## 3. Transform và Load vào Table Entity

Entity table chỉ lưu các loại entity types và mô tả (VD: MAJOR - "Chuyên ngành học")

In [3]:
# Định nghĩa mô tả cho các entity types
entity_type_descriptions = {
    "ORG": "ORGANIZATION : Tổ chức/Trường (ví dụ: Trường Đại học Bách khoa Hà Nội, Viện Công nghệ, các viện/khoa/bộ môn)",
    "PRO": "PROGRAM : Chương trình đào tạo (ví dụ: Chương trình Tiên tiến, Chương trình KSCLC, ELITECH)",
    "MAJ": "MAJOR : Ngành học, chuyên ngành, nghề nghiệp (ví dụ: Khoa học máy tính, Công nghệ thông tin, Kỹ thuật Điện tử - Viễn thông, Bác sĩ)",
    "SCO": "SCORE : Điểm số (ví dụ: 27.5 điểm, Điểm chuẩn 28)",
    "DATE": "DATE : Thời gian (ví dụ: Năm 2024, Năm 2025, 15/8/2024)",
    "EX": "EXAM : Kỳ thi (ví dụ: Kỳ thi đánh giá năng lực, TOEIC, IELTS, TOPIK)",
    "LOC": "LOCATION : Địa điểm (ví dụ: Hà Nội, Thành phố Hồ Chí Minh)",
    "FEE": "FEE : Học phí/Chi phí (ví dụ: Học phí, Mức đóng, Học bổng)",
    "SUBJ": "SUBJECT : Môn học (ví dụ: Toán, Lý, Hóa)",
    "TERM": "TERM : Tổ hợp môn (ví dụ: A1, B1)",
    "SAL": "SALARY : Lương (ví dụ: Mức lương, Thu nhập)"
}

# Parse NER labels để extract entity types
@udf(returnType=ArrayType(StringType()))
def extract_entity_types(ner_labels):
    """Extract unique entity types from NER labels"""
    if not ner_labels:
        return []
    
    labels = ner_labels.split()
    entity_types = set()
    
    for label in labels:
        if label.startswith('B-') or label.startswith('I-'):
            entity_type = label[2:]  # Lấy phần sau "B-" hoặc "I-"
            entity_types.add(entity_type)
    
    return list(entity_types)

# Extract unique entity types từ toàn bộ dữ liệu
df_entity_types_raw = df_result.select(
    explode(extract_entity_types(col("Label_NER"))).alias("entityType")
).filter(
    col("entityType").isNotNull()
).distinct()

# Tạo DataFrame với entityDetail từ mapping
from pyspark.sql.functions import lit

# Tạo list các entity types với descriptions
entity_list = []
for entity_type in df_entity_types_raw.collect():
    et = entity_type.entityType
    entity_list.append({
        "entityType": et,
        "entityDetail": entity_type_descriptions.get(et, f"Entity type: {et}")
    })

if entity_list:
    df_unique_entities = spark.createDataFrame(entity_list)
    
    # Thêm entityID
    window_spec = Window.orderBy("entityType")
    df_entities = df_unique_entities.withColumn(
        "entityID",
        row_number().over(window_spec)
    ).withColumn(
        "created_at", current_timestamp()
    ).withColumn(
        "updated_at", current_timestamp()
    ).select(
        col("entityID"),
        col("entityType"),
        col("entityDetail"),
        col("created_at"),
        col("updated_at")
    )
    
    print(f"\nSố lượng unique entity types từ dữ liệu mới: {df_entities.count()}")
    df_entities.show(20, truncate=False)
    
    # Kiểm tra và merge với entities đã có
    print("\n=== Loading data into Entity table ===")
    try:
        df_existing_entities = spark.table("nessie.gold_result_model_multi_task.Entity")
        existing_count = df_existing_entities.count()
        print(f"Tìm thấy {existing_count} entity types đã tồn tại")
        
        max_entity_id = df_existing_entities.agg({"entityID": "max"}).collect()[0][0]
        if max_entity_id is None:
            max_entity_id = 0
        print(f"Max entityID hiện tại: {max_entity_id}")
        
        # Chỉ thêm các entity types mới
        df_new_entities = df_entities.join(
            df_existing_entities.select("entityType"),
            on="entityType",
            how="left_anti"
        )
        
        new_count = df_new_entities.count()
        print(f"Số lượng entity types mới cần thêm: {new_count}")
        
        if new_count > 0:
            window_spec = Window.orderBy("entityType")
            df_new_entities = df_new_entities.withColumn(
                "entityID",
                row_number().over(window_spec) + max_entity_id
            ).select(
                col("entityID"),
                col("entityType"),
                col("entityDetail"),
                col("created_at"),
                col("updated_at")
            )
            
            df_new_entities.writeTo("nessie.gold_result_model_multi_task.Entity") \
                .using("iceberg") \
                .append()
            print(f"Đã thêm {new_count} entity types mới!")
            
            df_entities = df_existing_entities.union(df_new_entities)
        else:
            print("Không có entity types mới cần thêm")
            df_entities = df_existing_entities
            
    except Exception as e:
        print(f"Table Entity chưa có dữ liệu hoặc chưa tồn tại: {e}")
        print("Tạo mới table với dữ liệu hiện tại")
        df_entities.writeTo("nessie.gold_result_model_multi_task.Entity") \
            .using("iceberg") \
            .create()
        print(f"Đã tạo table Entity với {df_entities.count()} records!")
else:
    print("Không tìm thấy entity types trong dữ liệu")


Số lượng unique entity types từ dữ liệu mới: 12


+--------+----------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+
|entityID|entityType|entityDetail                                                                                                                       |created_at                |updated_at                |
+--------+----------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+
|1       |DATE      |DATE : Thời gian (ví dụ: Năm 2024, Năm 2025, 15/8/2024)                                                                            |2025-12-06 15:32:30.060096|2025-12-06 15:32:30.060096|
|2       |EX        |EXAM : Kỳ thi (ví dụ: Kỳ thi đánh giá năng lực, TOEIC, IELTS, TOPIK)                                                               |2025-12-06 15:3

Số lượng entity types mới cần thêm: 12


Đã thêm 12 entity types mới!


## 4. Transform và Load vào Table Topic

Topic table lưu các chủ đề và mô tả (VD: MAJOR - "Ngành học, chuyên ngành")

In [4]:
# Định nghĩa mô tả cho các topic types
topic_descriptions = {
    "MAJOR": "Ngành học, chuyên ngành",
    "SUBJECT_COMBINATION": "Tổ hợp môn thi (A, B, C, D)",
    "TUITION": "Học phí, học bổng",
    "CERTIFICATE": "IELTS, TOEFL, SAT, kỳ thi năng lực",
    "UNIVERSITY": "Thông tin trường",
    "STUDENT_LIFE": "Ký túc xá, CLB, làm thêm",
    "CAREER": "Nghề nghiệp, lương",
    "STUDY": "Phương pháp học, môn học ĐH",
    "LANGUAGE": "Tiếng Anh, ngoại ngữ",
    "OTHER": "Khác"
}

# Parse Label_Topic (multi-label, phân cách bằng '|')
df_topics_raw = df_result.select(
    explode(split(col("Label_Topic"), "\\|")).alias("topicName")
).filter(
    (col("topicName").isNotNull()) & 
    (col("topicName") != "None") &
    (trim(col("topicName")) != "")
).select(
    trim(col("topicName")).alias("topicName")
).distinct()

# Tạo list các topics với descriptions
topic_list = []
for topic in df_topics_raw.collect():
    topic_name = topic.topicName
    topic_list.append({
        "topicName": topic_name,
        "topicDetail": topic_descriptions.get(topic_name, "Chủ đề khác")
    })

if topic_list:
    df_unique_topics = spark.createDataFrame(topic_list)
    
    # Thêm topicID
    window_spec = Window.orderBy("topicName")
    df_topics = df_unique_topics.withColumn(
        "topicID",
        row_number().over(window_spec)
    ).withColumn(
        "created_at", current_timestamp()
    ).withColumn(
        "updated_at", current_timestamp()
    ).select(
        col("topicID"),
        col("topicName"),
        col("topicDetail"),
        col("created_at"),
        col("updated_at")
    )
    
    print(f"Số lượng unique topics từ dữ liệu mới: {df_topics.count()}")
    df_topics.show(20, truncate=False)
    
    print("\n=== Loading data into Topic table ===")
    try:
        df_existing_topics = spark.table("nessie.gold_result_model_multi_task.Topic")
        existing_count = df_existing_topics.count()
        print(f"Tìm thấy {existing_count} topics đã tồn tại")
        
        max_topic_id = df_existing_topics.agg({"topicID": "max"}).collect()[0][0]
        if max_topic_id is None:
            max_topic_id = 0
        print(f"Max topicID hiện tại: {max_topic_id}")
        
        # Chỉ thêm các topics mới
        df_new_topics = df_topics.join(
            df_existing_topics.select("topicName"),
            on="topicName",
            how="left_anti"
        )
        
        new_count = df_new_topics.count()
        print(f"Số lượng topics mới cần thêm: {new_count}")
        
        if new_count > 0:
            window_spec = Window.orderBy("topicName")
            df_new_topics = df_new_topics.withColumn(
                "topicID",
                row_number().over(window_spec) + max_topic_id
            ).select(
                col("topicID"),
                col("topicName"),
                col("topicDetail"),
                col("created_at"),
                col("updated_at")
            )
            
            df_new_topics.writeTo("nessie.gold_result_model_multi_task.Topic") \
                .using("iceberg") \
                .append()
            print(f"Đã thêm {new_count} topics mới!")
            
            df_topics = df_existing_topics.union(df_new_topics)
        else:
            print("Không có topics mới cần thêm")
            df_topics = df_existing_topics
            
    except Exception as e:
        print(f"Table Topic chưa có dữ liệu hoặc chưa tồn tại: {e}")
        print("Tạo mới table với dữ liệu hiện tại")
        df_topics.writeTo("nessie.gold_result_model_multi_task.Topic") \
            .using("iceberg") \
            .create()
        print(f"Đã tạo table Topic với {df_topics.count()} records!")
else:
    print("Không tìm thấy topics trong dữ liệu")

Số lượng unique topics từ dữ liệu mới: 10


+-------+-------------------+----------------------------------+--------------------------+--------------------------+
|topicID|topicName          |topicDetail                       |created_at                |updated_at                |
+-------+-------------------+----------------------------------+--------------------------+--------------------------+
|1      |CAREER             |Nghề nghiệp, lương                |2025-12-06 15:32:44.004228|2025-12-06 15:32:44.004228|
|2      |CERTIFICATE        |IELTS, TOEFL, SAT, kỳ thi năng lực|2025-12-06 15:32:44.004228|2025-12-06 15:32:44.004228|
|3      |LANGUAGE           |Tiếng Anh, ngoại ngữ              |2025-12-06 15:32:44.004228|2025-12-06 15:32:44.004228|
|4      |MAJOR              |Ngành học, chuyên ngành           |2025-12-06 15:32:44.004228|2025-12-06 15:32:44.004228|
|5      |OTHER              |Khác                              |2025-12-06 15:32:44.004228|2025-12-06 15:32:44.004228|
|6      |STUDENT_LIFE       |Ký túc xá, CLB, làm

Số lượng topics mới cần thêm: 10


[Stage 41:=================================================>      (44 + 6) / 50]

Đã thêm 10 topics mới!


## 5. Transform và Load vào Table Post

In [5]:
# Tạo table Post từ result model
df_posts = df_result.select(
    col("postID"),
    col("description_Normalized").alias("description"),
    col("timePublish"),
    col("likeCount"),
    col("commentCount"),
    col("shareCount"),
    col("Label_Intent").alias("intent"),
    col("type")
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
)

total_posts = df_posts.count()
print(f"Tổng số posts mới: {total_posts}")
df_posts.show(5, truncate=80)

# Load vào table Post với batch processing để tránh OOM
print("\n=== Loading data into Post table ===")
if total_posts > 0:
    # Repartition để tối ưu memory
    df_posts = df_posts.repartition(20)
    
    try:
        # Batch processing: Xử lý 1000 records mỗi lần
        BATCH_SIZE = 1000
        num_batches = (total_posts + BATCH_SIZE - 1) // BATCH_SIZE
        
        if num_batches > 1:
            print(f"Xử lý {num_batches} batches (mỗi batch ~{BATCH_SIZE} records)...")
            
            # Lấy danh sách postID để chia batch
            post_ids = [row.postID for row in df_posts.select("postID").collect()]
            
            for i in range(num_batches):
                start_idx = i * BATCH_SIZE
                end_idx = min((i + 1) * BATCH_SIZE, total_posts)
                batch_ids = post_ids[start_idx:end_idx]
                
                print(f"  Batch {i+1}/{num_batches}: {len(batch_ids)} records...", end=" ")
                
                # Filter batch
                df_batch = df_posts.filter(col("postID").isin(batch_ids))
                
                # Write batch
                if i == 0:
                    try:
                        df_batch.writeTo("nessie.gold_result_model_multi_task.Post") \
                            .using("iceberg") \
                            .append()
                        print("OK")
                    except Exception as e:
                        if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
                            df_batch.writeTo("nessie.gold_result_model_multi_task.Post") \
                                .using("iceberg") \
                                .create()
                            print("OK (created)")
                        else:
                            raise e
                else:
                    df_batch.writeTo("nessie.gold_result_model_multi_task.Post") \
                        .using("iceberg") \
                        .append()
                    print("OK")
            
            print(f"\nĐã thêm tổng cộng {total_posts} posts!")
        else:
            # Ít hơn 1000 records, write trực tiếp
            print("Số lượng nhỏ, write trực tiếp...")
            try:
                df_posts.writeTo("nessie.gold_result_model_multi_task.Post") \
                    .using("iceberg") \
                    .append()
                print(f"Đã thêm {total_posts} posts!")
            except Exception as e:
                if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
                    df_posts.writeTo("nessie.gold_result_model_multi_task.Post") \
                        .using("iceberg") \
                        .create()
                    print(f"Đã tạo table Post với {total_posts} records!")
                else:
                    raise e
                    
    except Exception as e:
        print(f"\nLỗi: {e}")
        raise e
else:
    print("Không có posts mới cần thêm")


Tổng số posts mới: 3295
+----------------------------------------+--------------------------------------------------------------------------------+-------------------+---------+------------+----------+----------+--------+--------------------------+--------------------------+
|                                  postID|                                                                     description|        timePublish|likeCount|commentCount|shareCount|    intent|    type|                created_at|                updated_at|
+----------------------------------------+--------------------------------------------------------------------------------+-------------------+---------+------------+----------+----------+--------+--------------------------+--------------------------+
|@eduquiz.study/video/7475915684271197448|                                                Những ngành siêu hot của khối D.|2025-02-27 00:00:00|    10200|         254|         0|share_info|  TikTok|2025-12-06 15:32:49.633

  Batch 1/4: 1000 records... 

OK
  Batch 2/4: 1000 records... 

OK
  Batch 3/4: 1000 records... 

OK
  Batch 4/4: 295 records... 

[Stage 74:>                                                         (0 + 1) / 1]

OK

Đã thêm tổng cộng 3295 posts!


## 6. Transform và Load vào Table Post_Entity

Bảng quan hệ M:N giữa Post và Entity - Lưu các entity values thực tế được extract từ posts

In [7]:
# Parse NER labels để extract entities với postID
@udf(returnType=ArrayType(StructType([
    StructField("entityType", StringType()),
    StructField("entityName", StringType())
])))
def extract_entities_with_values(ner_labels, text):
    """Extract entities từ NER labels và text - trả về entityType và giá trị thực tế"""
    if not ner_labels or not text:
        return []
    
    labels = ner_labels.split()
    words = text.split()
    
    if len(labels) != len(words):
        return []
    
    entities = []
    current_type = None
    current_words = []
    
    for label, word in zip(labels, words):
        if label.startswith('B-'):
            # Kết thúc entity trước đó (nếu có)
            if current_type and current_words:
                entities.append({
                    'entityType': current_type,
                    'entityName': ' '.join(current_words)
                })
            # Bắt đầu entity mới
            current_type = label[2:]
            current_words = [word]
        elif label.startswith('I-') and current_type:
            # Tiếp tục entity hiện tại
            if label[2:] == current_type:
                current_words.append(word)
        else:
            # Kết thúc entity
            if current_type and current_words:
                entities.append({
                    'entityType': current_type,
                    'entityName': ' '.join(current_words)
                })
            current_type = None
            current_words = []
    
    # Xử lý entity cuối cùng
    if current_type and current_words:
        entities.append({
            'entityType': current_type,
            'entityName': ' '.join(current_words)
        })
    
    return entities

# Extract entities với values từ posts
df_post_entities_raw = df_result.select(
    col("postID"),
    explode(extract_entities_with_values(col("Label_NER"), col("description_Normalized"))).alias("entity")
).select(
    col("postID"),
    col("entity.entityType").alias("entityType"),
    col("entity.entityName").alias("entityName")
).filter(
    (col("entityType").isNotNull()) & (col("entityName").isNotNull())
)

print(f"\nTổng số entity mentions được extract: {df_post_entities_raw.count()}")
df_post_entities_raw.show(10, truncate=False)

# Add entity order trong mỗi post
window_spec = Window.partitionBy("postID").orderBy("entityType", "entityName")
df_post_entities_ordered = df_post_entities_raw.withColumn(
    "entityOrder",
    row_number().over(window_spec)
)

# Join với Entity table để lấy entityID (chỉ cần join theo entityType)
df_post_entity = df_post_entities_ordered.join(
    df_entities.select("entityID", "entityType"),
    on="entityType",
    how="inner"
).select(
    col("postID"),
    col("entityID"),
    col("entityOrder"),
    col("entityName")
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
)

print(f"\nTổng số quan hệ Post-Entity mới: {df_post_entity.count()}")
print("\nSample data với entity names:")
df_post_entity.show(10, truncate=False)

print("\n=== Loading data into Post_Entity table ===")
if df_post_entity.count() > 0:
    try:
        df_post_entity.writeTo("nessie.gold_result_model_multi_task.Post_Entity") \
            .using("iceberg") \
            .append()
        print(f"Đã thêm {df_post_entity.count()} quan hệ Post-Entity mới!")
        
    except Exception as e:
        if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
            print("Table Post_Entity chưa tồn tại, tạo mới...")
            df_post_entity.writeTo("nessie.gold_result_model_multi_task.Post_Entity") \
                .using("iceberg") \
                .create()
            print(f"Đã tạo table Post_Entity với {df_post_entity.count()} records!")
        else:
            raise e
else:
    print("Không có quan hệ Post-Entity mới cần thêm")


Tổng số entity mentions được extract: 17876
+-----------------------------------------+----------+------------------------------------+
|postID                                   |entityType|entityName                          |
+-----------------------------------------+----------+------------------------------------+
|@eduquiz.study/video/7475915684271197448 |TERM      |khối D.                             |
|2033261754197597                         |DATE      |2007                                |
|2033261754197597                         |MAJ       |sư_phạm                             |
|2033564457500660                         |MISC      |thí_sinh                            |
|2033564457500660                         |ORG       |Đại_học Y_Dược Thành_phố Hồ_Chí_Minh|
|2033525434171229                         |ORG       |Bộ Giáo_dục và Đào_tạo              |
|@nguyenvuongtkm/video/7289587753388494082|MAJ       |Nghề cơ_khí                         |
|@nguyenvuongtkm/video/728958775338


Tổng số quan hệ Post-Entity mới: 17876

Sample data với entity names:


+--------------------------------------------+--------+-----------+----------+--------------------------+--------------------------+
|postID                                      |entityID|entityOrder|entityName|created_at                |updated_at                |
+--------------------------------------------+--------+-----------+----------+--------------------------+--------------------------+
|@zing_studio/video/7260303093034700050      |1       |1          |năm 2 02  |2025-12-06 15:42:56.671529|2025-12-06 15:42:56.671529|
|@zing_studio/video/7260070959418215681      |1       |2          |2023      |2025-12-06 15:42:56.671529|2025-12-06 15:42:56.671529|
|@zing_studio/video/7260070959418215681      |1       |1          |2022      |2025-12-06 15:42:56.671529|2025-12-06 15:42:56.671529|
|@yuujapan.vn/video/7571413470390881556      |1       |1          |2025      |2025-12-06 15:42:56.671529|2025-12-06 15:42:56.671529|
|@yte247.news/video/7452188748042816786      |1       |3          |28

[Stage 123:===============================================>       (44 + 6) / 51]

Đã thêm 17876 quan hệ Post-Entity mới!


## 7. Transform và Load vào Table Post_Topic

Bảng quan hệ M:N giữa Post và Topic

In [8]:
# Parse topics cho mỗi post
df_post_topics_raw = df_result.select(
    col("postID"),
    explode(split(col("Label_Topic"), "\\|")).alias("topicName")
).filter(
    (col("topicName").isNotNull()) & 
    (col("topicName") != "None") &
    (trim(col("topicName")) != "")
).select(
    col("postID"),
    trim(col("topicName")).alias("topicName")
)

# Join với Topic table để lấy topicID
df_post_topic = df_post_topics_raw.join(
    df_topics.select("topicID", "topicName"),
    on="topicName",
    how="inner"
).select(
    col("postID"),
    col("topicID")
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
)

print(f"Tổng số quan hệ Post-Topic mới: {df_post_topic.count()}")
df_post_topic.show(10, truncate=False)

print("\n=== Loading data into Post_Topic table ===")
if df_post_topic.count() > 0:
    try:
        df_post_topic.writeTo("nessie.gold_result_model_multi_task.Post_Topic") \
            .using("iceberg") \
            .append()
        print(f"Đã thêm {df_post_topic.count()} quan hệ Post-Topic mới!")
    except Exception as e:
        if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
            print("Table Post_Topic chưa tồn tại, tạo mới...")
            df_post_topic.writeTo("nessie.gold_result_model_multi_task.Post_Topic") \
                .using("iceberg") \
                .create()
            print(f"Đã tạo table Post_Topic với {df_post_topic.count()} records!")
        else:
            raise e
else:
    print("Không có quan hệ Post-Topic mới cần thêm")


Tổng số quan hệ Post-Topic mới: 5023


+-----------------------------------------+-------+--------------------------+--------------------------+
|postID                                   |topicID|created_at                |updated_at                |
+-----------------------------------------+-------+--------------------------+--------------------------+
|@mai_sun/video/7374948151045754128       |1      |2025-12-06 15:44:01.072513|2025-12-06 15:44:01.072513|
|@truyenthongtdd/video/7374411385733696769|1      |2025-12-06 15:44:01.072513|2025-12-06 15:44:01.072513|
|1794881368035638                         |1      |2025-12-06 15:44:01.072513|2025-12-06 15:44:01.072513|
|1796789334511508                         |1      |2025-12-06 15:44:01.072513|2025-12-06 15:44:01.072513|
|1796374157886359                         |1      |2025-12-06 15:44:01.072513|2025-12-06 15:44:01.072513|
|2113562269500878                         |1      |2025-12-06 15:44:01.072513|2025-12-06 15:44:01.072513|
|2116258409231264                         |1  

[Stage 146:==============================================>        (43 + 6) / 51]

Đã thêm 5023 quan hệ Post-Topic mới!


## 8. Verify Dữ Liệu Đã Load

In [9]:
print("="*80)
print("SUMMARY - Data Loaded to Gold Tables")
print("="*80)

tables = [
    "Entity",
    "Topic",
    "Post",
    "Post_Entity",
    "Post_Topic"
]

for table_name in tables:
    df_verify = spark.table(f"nessie.gold_result_model_multi_task.{table_name}")
    count = df_verify.count()
    print(f"\n{'='*80}")
    print(f"Table: {table_name}")
    print(f"Total records: {count}")
    print(f"{'='*80}")
    df_verify.show(5, truncate=False)

print("\n" + "="*80)
print("Hoàn thành load dữ liệu vào tất cả Gold tables!")
print("="*80)

SUMMARY - Data Loaded to Gold Tables

Table: Entity
Total records: 12
+--------+----------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+
|entityID|entityType|entityDetail                                                                                                                       |created_at                |updated_at                |
+--------+----------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+
|1       |DATE      |DATE : Thời gian (ví dụ: Năm 2024, Năm 2025, 15/8/2024)                                                                            |2025-12-06 15:32:34.117264|2025-12-06 15:32:34.117264|
|2       |EX        |EXAM : Kỳ thi (ví dụ: Kỳ thi đánh giá năng lực, TOEIC, IELTS, TOPIK)         

## 9. Dừng Spark Session

In [11]:
# Dừng Spark Session
spark.stop()
print("Spark Session đã được dừng!")


Spark Session đã được dừng!
